In [2]:
import os
import pandas

In [3]:
print(os.getcwd())

/Users/mf019/bioinformatics/longread_GWAS/notebooks/metadata/pair_finding


In [4]:
v5_file = '../wrangled/longread_metadata_v5.csv'
v6_file = '../wrangled/longread_metadata_v6.csv'
metadata_v5 = pandas.read_csv(v5_file)
metadata_v5.head()

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type
0,5A18NP1-JBb08-A,1,NaN,culture,US,NY,NaN,NaN,nanopore,A,AT1
1,UNY147,34,3.0,NaN,US,NY,NaN,L,illumina,J,NaN
2,5A18NP1-JBb08-C,1,NaN,culture,US,NY,NaN,NaN,nanopore,A,AT1
3,5A18NP1-JBb08-B,1,NaN,culture,US,NY,NaN,NaN,nanopore,A,AT1
4,5A18NP1-JBb08-E,1,NaN,culture,US,NY,NaN,NaN,nanopore,A,AT1


# Now to fix the hybrid and pacbio assembly naming!

Nathan showed me that the hybrids and pacbio assemblies have metadata and a renaming scheme at the very bottom of the jupyter renaming sheet. 

I will first import that table (jupyter renaming scheme)
create a lookup on column A and B,
then import the list of assemblies (filenames sans extensions),
for each name that is present in col B, add the corresponding value from col A to a new column.
download files from bucket
rename files as needed
then re-upload to bucket
then subset assemblies based on method and pairs.
then run roary
then annotate pangenes
then hand off to rest of team.


In [11]:
jupyter_renaming_table_file = '/Users/mf019/bioinformatics/longread_GWAS/notebooks/metadata/lymeseq_original_sheets/LymeSeq_SampleTrack - Renaming Scheme Table for Jupyter (1).csv'
jupyter_table = pandas.read_csv(jupyter_renaming_table_file)
print(jupyter_table.shape)
print(jupyter_table.head())

(1215, 54)
  Rename_A Original_Name Origin   Location OspC_Type  RST_Type OspC_Allele  \
0     UMA1            N2     US  Nantucket         K       2.0         NaN   
1     UMA2            N4     US  Nantucket         F       2.0         NaN   
2     UMA3            N7     US  Nantucket         K       2.0         NaN   
3     UMA4            N9     US  Nantucket         K       2.0         NaN   
4     UMA5           N11     US  Nantucket         N       2.0         NaN   

  MLST Disseminated BC + (Y/N)  ... Source.1 Code     Genus    Species.1  \
0    3            L         ND  ...    Jacob  NaN  Borrelia  burgdorferi   
1    8            L         ND  ...    Jacob  NaN  Borrelia  burgdorferi   
2    3            L         ND  ...    Jacob  NaN  Borrelia  burgdorferi   
3    3            L         ND  ...    Jacob  NaN  Borrelia  burgdorferi   
4   36            L         ND  ...    Jacob  NaN  Borrelia  burgdorferi   

   FastQC_1  BBDuk  FastQC_2 Spades BBMerge sp_identification  

In [12]:

renaming_map = jupyter_table[['Rename_A','Original_Name']]
print(renaming_map.shape)
print(renaming_map.head())


(1215, 2)
  Rename_A Original_Name
0     UMA1            N2
1     UMA2            N4
2     UMA3            N7
3     UMA4            N9
4     UMA5           N11


In [13]:
renaming_dict = {}
for index, row in renaming_map.iterrows():
    rename = renaming_map.at[index, 'Rename_A']
    original = renaming_map.at[index, 'Original_Name']
    renaming_dict[original] = rename


In [14]:
nanopore_assemblies = metadata_v5[metadata_v5['method'] == 'nanopore']
pacbio_assemblies = metadata_v5[metadata_v5['method'] == 'pacbio']
hybrid_assemblies = metadata_v5[metadata_v5['method'] == 'hybrid']
illumina_assemblies = metadata_v5[metadata_v5['method'] == 'illumina']
print(len(nanopore_assemblies))
print(len(pacbio_assemblies))
print(len(hybrid_assemblies))
print(len(illumina_assemblies))

13
19
47
299


In [17]:
pacbio_assemblies

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type
64,MR616,14,3.0,Skin,US,NY,NaN,L,pacbio,G,AT7
93,MC149,221,NaN,Skin,US,WI,NaN,L,pacbio,O,AT3
124,B348,19,3.0,Skin,US,NY,NaN,D,pacbio,E,AT5
142,B331,16,3.0,Skin,US,NY,NaN,L,pacbio,I,AT9
144,BL522,12,3.0,NaN,NY,NaN,NaN,D,pacbio,M,AT12
146,B418,19,3.0,Skin,US,NY,BL252,D,pacbio,E,AT5
158,B356,12,3.0,Skin,US,NY,NaN,L,pacbio,M,AT12
171,B483,9,2.0,NaN,NY,NaN,NaN,ND,pacbio,N,AT13
188,MC104,4,NaN,CSF,US,WI,NaN,D,pacbio,H,AT8
193,B477,38,3.0,Skin,US,NY,NaN,L,pacbio,D,NaN


In [16]:
for index, row in pacbio_assemblies.iterrows():
    newname = row['newname'] + 'P'
    #print(str(newname))
    if newname in renaming_dict:
        pacbio_assemblies.at[index, 'paired_sample'] = renaming_dict[newname].str[:-1]
    else:
        print(f"NOT FOUND! {newname}")

#print(pacbio_assemblies)

for index, row in hybrid_assemblies.iterrows():
    hybrid_assemblies.at[index, 'newname'] = row['Isolate'] + 'H'

for index, row in hybrid_assemblies.iterrows():
    newname = row['newname']
    #print(str(newname))
    if newname in renaming_dict:
        hybrid_assemblies.at[index, 'paired_sample'] = renaming_dict[newname].str[:-1]
    else:
        print(f"NOT FOUND! {newname}")


KeyError: 'newname'

In [150]:
# Create a list of tables to merge
tables_to_merge = [pacbio_assemblies, hybrid_assemblies, illumina_assemblies]

# Merge the tables using pd.concat()
merged_table = pandas.concat(tables_to_merge)

# Print the merged table
print(merged_table)

    Isolate MLST  RST_Type Source Origin   Location skin/blood_pairs  \
64    MR616   14       3.0   Skin     US         NY              NaN   
93    MC149  221       NaN   Skin     US         WI              NaN   
124    B348   19       3.0   Skin     US         NY              NaN   
142    B331   16       3.0   Skin     US         NY              NaN   
144   BL522   12       3.0    NaN     NY        NaN              NaN   
..      ...  ...       ...    ...    ...        ...              ...   
347   URI19   59       1.0    NaN     US         RI              NaN   
348   UMA13   59       1.0    NaN     US  Nantucket              NaN   
349    UMA2    8       2.0    NaN     US  Nantucket              NaN   
356   URI84   59       2.0    NaN     US         RI              NaN   
357   URI20   14       3.0    NaN     US         RI              NaN   

    Disseminated    method OspC_Type AT_type newname paired_sample  
64             L    pacbio         G     AT7  MR616P       UNY203P

In [154]:
merged_table.to_csv('metadata_v6', index=False)

In [151]:
print(merged_table.loc[merged_table['skin/blood_pairs'].notnull()])

    Isolate MLST  RST_Type Source Origin Location skin/blood_pairs  \
146    B418   19       3.0   Skin     US       NY            BL252   
290    B247    3       2.0   Skin     US       NY            BL146   

    Disseminated  method OspC_Type AT_type newname paired_sample  
146            D  pacbio         E     AT5   B418P           NaN  
290            D  pacbio         K    AT11   B247P       UNY149P  


In [215]:
repeated_seqs_table = '/Users/mf019/bioinformatics/longread_GWAS/notebooks/metadata/lymeseq_original_sheets/LymeSeq_SampleTrack - Repeated Sequence Sheet.csv'
repeated_seqs_df = pandas.read_csv(repeated_seqs_table)
#repeated_seqs_df = repeated_seqs_df.drop(axis=1, labels='Unnamed: 2')
repeated_seqs_df1 = repeated_seqs_df[['Repeated', 'Original_Name_1']]
repeated_seqs_df2 = repeated_seqs_df[['Skin/Blood Pairs', 'Original_Name_2']]
repeated_seqs_df1 = repeated_seqs_df1.dropna()
print(repeated_seqs_df1)
print(repeated_seqs_df2)

   Repeated Original_Name_1
0      UMA1              N2
1      UMA2              N4
2      UMA3              N7
3      UMA4              N9
4      UMA5             N11
5      UMA7             N15
6      UMA8             N17
7     UMA11             N21
8     UMA14             N30
9     UMA15             N31
10    UMA16             N32
11    URI17              R5
12    URI18              R6
13    URI19              R7
14    URI20              R8
15    URI21              R9
16    URI22             R10
17    URI23             R11
18    URI24             R12
   Skin/Blood Pairs Original_Name_2
0      UNY133(BL55)             B48
1      UNY134(BL72)             B62
2     UNY149(BL146)            B247
3     UNY149(BL149)            B250
4     UNY153(BL152)            B253
5     UNY176(BL220)            B373
6     UNY177(BL224)            B379
7     UNY178(BL229)            B385
8     UNY179(BL231)            B388
9     UNY181(BL241)            B400
10    UNY182(BL242)            B401
11    UN

In [216]:
repeated_seqs_df2[['renamed_pair', 'original_name_pair']] = repeated_seqs_df2['Skin/Blood Pairs'].str.split('(', expand=True)
repeated_seqs_df2['original_name_pair'] = repeated_seqs_df2['original_name_pair'].str.rstrip(')')
repeated_seqs_df2

/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_60919/2191308876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repeated_seqs_df2[['renamed_pair', 'original_name_pair']] = repeated_seqs_df2['Skin/Blood Pairs'].str.split('(', expand=True)
/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_60919/2191308876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repeated_seqs_df2[['renamed_pair', 'original_name_pair']] = repeated_seqs_df2['Skin/Blood Pairs'].str.split('(', expand=True)
/var

,Skin/Blood Pairs,Original_Name_2,renamed_pair,original_name_pair
0,UNY133(BL55),B48,UNY133,BL55
1,UNY134(BL72),B62,UNY134,BL72
2,UNY149(BL146),B247,UNY149,BL146
3,UNY149(BL149),B250,UNY149,BL149
4,UNY153(BL152),B253,UNY153,BL152
5,UNY176(BL220),B373,UNY176,BL220
6,UNY177(BL224),B379,UNY177,BL224
7,UNY178(BL229),B385,UNY178,BL229
8,UNY179(BL231),B388,UNY179,BL231
9,UNY181(BL241),B400,UNY181,BL241


In [321]:
# okay now we need to pull the isolate/sample ids from the notes column
wgs_table_file = '/Users/mf019/bioinformatics/longread_GWAS/notebooks/metadata/lymeseq_original_sheets/LymeSeq_SampleTrack - Whole Genome Sequenced Table (1).csv'
wgs_table_df = pandas.read_csv(wgs_table_file)

In [327]:
wgs_table_df['Skin/Blood Pairs'] = wgs_table_df['Notes'].str.split(' ').str[-1]
wgs_table_filtered_df = wgs_table_df[['Rename_A','Original_Name', 'Source', 'Seq Method?', 'OspC_Type', 'MLST', 'Disseminated', 'Skin/Blood Pairs']]
print(wgs_table_df[wgs_table_df['Skin/Blood Pairs'].notna()])


    New Rename? Rename_A Seq Method? Original_Name Source OspC_Type  MLST  \
47          NaN      NaN      PacBio         BL206  Blood         A   1.0   
49          NaN      NaN      PacBio         BL515  Blood         C  11.0   
50          NaN      NaN      PacBio         BL522  Blood         M  12.0   
51          NaN      NaN      PacBio         BL546  Blood         H   4.0   
56          NaN      NaN      PacBio         MR614   Skin         C  11.0   
58          NaN   UNY208      PacBio         MR641   Skin         H   4.0   
64          NaN      NaN      PacBio         BL224  Blood         K   3.0   

   Disseminated                   Notes Skin/Blood Pairs  
47            D  Skin isolate is UNY170           UNY170  
49            D         Pair with MR614            MR614  
50            D  Skin isolate is UNY204           UNY204  
51            D         Pair with MR641            MR641  
56            D         Pair with BL515            BL515  
58            D         Pair 

In [328]:
print(wgs_table_filtered_df['Original_Name'].map(renaming_dict))

0         NaN
1         NaN
2         NaN
3       UCT29
4       UCT30
       ...   
61     UNY172
62        NaN
63     UNY193
64    UNY1038
65        NaN
Name: Original_Name, Length: 66, dtype: object


In [329]:
import glob
# Okay maybe that helped some?
# Turns out that the repeated sequence sheet is actually two tables in a trench coat.
assembly_dir = '/Users/mf019/bioinformatics/longread_GWAS/assemblies'
list_of_hybrids = glob.glob(f'{assembly_dir}/hybrid/contigs/*.fasta')
list_of_pacbio = glob.glob(f'{assembly_dir}/pacbio/contigs/*.fasta')
print(len(list_of_hybrids))
print(len(list_of_pacbio))

47
20


In [438]:
hybrid_asm_naming_df = pandas.DataFrame(columns=['original', 'renamed'])
hybrid_names = []
for asm in list_of_hybrids:
    filename = asm.split('/')[-1].split('.')[0] + "H"
    if filename in renaming_dict:
        rename = renaming_dict[filename]
    else:
        rename = filename
    hybrid_asm_naming_df.loc[len(hybrid_asm_naming_df)] = [filename, rename]
    hybrid_names.append(rename)
    print(f'file:{filename}, rename:{rename}')

pacbio_asm_naming_df = pandas.DataFrame(columns=['original', 'renamed'])
pacbio_names = []
for asm in list_of_pacbio:
    filename = asm.split('/')[-1].split('.')[0] + "P"
    if filename in renaming_dict:
        rename = renaming_dict[filename]
    else:
        rename = filename
    pacbio_asm_naming_df.loc[len(pacbio_asm_naming_df)] = [filename, rename]
    pacbio_names.append(rename)
    print(f'file:{filename}, rename:{rename}')

file:K7H, rename:URI42H
file:K60H, rename:URI87H
file:K25H, rename:URI36H
file:K71H, rename:URI91H
file:K11H, rename:UCT30H
file:K22H, rename:URI34H
file:K19H, rename:URI48H
file:K76H, rename:UCT113H
file:K16H, rename:URI33H
file:K82H, rename:URI117H
file:K10H, rename:UCT31H
file:K84H, rename:UCT96H
file:K70H, rename:UCT110H
file:K24H, rename:UCT35H
file:K6H, rename:URI41H
file:K17H, rename:URI46H
file:K83H, rename:URI118H
file:K69H, rename:UCT109H
file:EU14_B89613H, rename:ESI403H
file:L113H, rename:ESI425H
file:K18H, rename:URI47H
file:K1H, rename:UCT29H
file:K66H, rename:URI107H
file:K80H, rename:UCT95H
file:K74H, rename:URI111H
file:K2H, rename:URI39H
file:K13H, rename:URI45H
file:K56H, rename:URI86H
file:K73H, rename:URI93H
file:K59H, rename:URI103H
file:EU57_B190413H, rename:ESI26H
file:K62H, rename:URI88H
file:K3H, rename:URI40H
file:K64H, rename:URI89H
file:K21H, rename:UCT50H
file:K90H, rename:UCT124H
file:XD1H, rename:XYZ459H
file:EU51_B192411H, rename:ESI361
file:K30H, renam

In [439]:
hybrid_asm_naming_df
hybrid_rename_sed_script = ""
for index, row in hybrid_asm_naming_df.iterrows():
    original_name = f"{row['original']}.hybrid.assembly.contigs.fasta"
    renamed_name = f"{row['renamed']}.fasta"
    hybrid_rename_sed_script += f"s/{original_name}/{renamed_name}/g\n"

with open('hybrid_rename_script.sed', 'w') as file:
    file.write(hybrid_rename_sed_script)


In [440]:
pacbio_asm_naming_df
pacbio_rename_sed_script = ""
for index, row in pacbio_asm_naming_df.iterrows():
    original_name = f"{row['original']}.pacbio.assembly.contigs.fasta"
    renamed_name = f"{row['renamed']}.fasta"
    pacbio_rename_sed_script += f"s/{original_name}/{renamed_name}/g\n"

with open('pacbio_rename_script.sed', 'w') as file:
    file.write(pacbio_rename_sed_script)

In [441]:
# Ok now I need to parse the jupyter renaming table as well as the rest of everything to identify the paired assemblies
# and then separate those out into a single final table....
# Then I can deal with the renaming to ensure that the same sample is compared properly.
# Example: UNY123 and UNY456 are blood/skin pairs from the same infection.
# Lets also say those were redone with pacbio,
# thus, UNY123_B_I and UNY123_S_I
# and , UNY123_B_P and UNY123_S_P ??
# Does this make sense?

#What do I have?
- [x] list of assemblies used (filenames)
- [x] list of pacbio assemblies and their corresponding names from metadata
- [x] list of hybrid assemblies and their corresponding names from metadata
- [x] bunch of metadata for all illumina assemblies
- [x] list of blood/skin pairs (from metadata)
- [x] list of resequenced samples (from metadata)

In [442]:
print(f'pacbio assemblies: {len(pacbio_names)}')
print(pacbio_names)
print(f'hybrid assemblies: {len(hybrid_names)}')
print(hybrid_names)

pacbio assemblies: 20
['UNY149P', 'UNY193P', 'UWI283P', 'UNY1128P', 'UNY1090P', 'UNY203P', 'UNY1085P', 'UNY169P', 'UNY1083P', 'B500P', 'UWI248P', 'UNY990P', 'UWI263P', 'UNY208P', 'UNY1032P', 'B331P', 'UWI247P', 'UNY1038P', 'B418P', 'UNY172P']
hybrid assemblies: 47
['URI42H', 'URI87H', 'URI36H', 'URI91H', 'UCT30H', 'URI34H', 'URI48H', 'UCT113H', 'URI33H', 'URI117H', 'UCT31H', 'UCT96H', 'UCT110H', 'UCT35H', 'URI41H', 'URI46H', 'URI118H', 'UCT109H', 'ESI403H', 'ESI425H', 'URI47H', 'UCT29H', 'URI107H', 'UCT95H', 'URI111H', 'URI39H', 'URI45H', 'URI86H', 'URI93H', 'URI103H', 'ESI26H', 'URI88H', 'URI40H', 'URI89H', 'UCT50H', 'UCT124H', 'XYZ459H', 'ESI361', 'URI56H', 'URI112H', 'UCT32H', 'K89H', 'URI102H', 'UCT92H', 'URI120H', 'URI44H', 'URI101H']


In [443]:
# ok so now lets pull in that jupyter table and pull all of the samples in...
renaming_scheme_jupyter_file = '/Users/mf019/bioinformatics/longread_GWAS/notebooks/metadata/lymeseq_original_sheets/LymeSeq_SampleTrack - Renaming Scheme Table for Jupyter (1).csv'
jupyter_df = pandas.read_csv(renaming_scheme_jupyter_file)

In [444]:
filtered_jupyter_df = jupyter_df[jupyter_df['Rename_A'].isin(pacbio_names) | jupyter_df['Rename_A'].isin(hybrid_names)]
filtered_jupyter_df['original_pair_name'] = filtered_jupyter_df['Original_Name'].str[:-1:]
filtered_jupyter_df['renamed_pair_name'] = filtered_jupyter_df['Rename_A'].str[:-1:]
renamed_names_pbh = list(filtered_jupyter_df['renamed_pair_name'])
original_names_pbh = list(filtered_jupyter_df['original_pair_name'])

/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_60919/2462978438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_jupyter_df['original_pair_name'] = filtered_jupyter_df['Original_Name'].str[:-1:]
/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_60919/2462978438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_jupyter_df['renamed_pair_name'] = filtered_jupyter_df['Rename_A'].str[:-1:]


In [445]:
filtered_table1 = merged_table[merged_table['Isolate'].isin(renamed_names_pbh) | merged_table['Isolate'].isin(original_names_pbh)]
filtered_table1

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type,newname,paired_sample
64,MR616,14,3.0,Skin,US,NY,NaN,L,pacbio,G,AT7,MR616P,UNY203P
93,MC149,221,NaN,Skin,US,WI,NaN,L,pacbio,O,AT3,MC149P,UWI283P
124,B348,19,3.0,Skin,US,NY,NaN,D,pacbio,E,AT5,B348P,UNY169P
144,BL522,12,3.0,NaN,NY,NaN,NaN,D,pacbio,M,AT12,BL522P,UNY1085P
158,B356,12,3.0,Skin,US,NY,NaN,L,pacbio,M,AT12,B356P,UNY172P
...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,URI111,1,1.0,NaN,US,RI,NaN,D,illumina,A,AT1,NaN,NaN
325,URI91,1,1.0,NaN,US,RI,NaN,L,illumina,A,AT1,NaN,NaN
341,URI88,4,2.0,NaN,US,RI,NaN,L,illumina,H,AT8,NaN,NaN
342,URI101,1,1.0,NaN,US,RI,NaN,D,illumina,A,AT1,NaN,NaN


In [446]:
print(len(list_of_hybrids))
print(len(list_of_pacbio))

47
20


In [448]:
metadata_v6_df = pandas.read_csv('/Users/mf019/bioinformatics/longread_GWAS/notebooks/metadata/pair_finding/metadata_v6')
metadata_v6_df

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type,newname,paired_sample
0,MR616,14,3.0,Skin,US,NY,NaN,L,pacbio,G,AT7,MR616P,UNY203P
1,MC149,221,NaN,Skin,US,WI,NaN,L,pacbio,O,AT3,MC149P,UWI283P
2,B348,19,3.0,Skin,US,NY,NaN,D,pacbio,E,AT5,B348P,UNY169P
3,B331,16,3.0,Skin,US,NY,NaN,L,pacbio,I,AT9,B331P,NaN
4,BL522,12,3.0,NaN,NY,NaN,NaN,D,pacbio,M,AT12,BL522P,UNY1085P
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,URI19,59,1.0,NaN,US,RI,NaN,L,illumina,B,AT2,NaN,NaN
361,UMA13,59,1.0,NaN,US,Nantucket,NaN,L,illumina,B,NaN,NaN,NaN
362,UMA2,8,2.0,NaN,US,Nantucket,NaN,L,illumina,F,AT6,NaN,NaN
363,URI84,59,2.0,NaN,US,RI,NaN,L,illumina,B,NaN,NaN,NaN


In [453]:
subset1 = metadata_v6_df[metadata_v6_df['method'].isin(['pacbio', 'hybrid'])]
subset1a = jupyter_df[jupyter_df['Rename_A'].isin(subset1['paired_sample'])]
subset1b = jupyter_df[jupyter_df['Rename_A'].isin(subset1['paired_sample'].str[:-1])]
